In [14]:
!pip -q install lxml rapidfuzz pyspellchecker

import os, re
import numpy as np
import pandas as pd
from datetime import datetime, timezone

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

RUN_AT_UTC = datetime.now(timezone.utc).isoformat()
print("Run at (UTC):", RUN_AT_UTC)

Run at (UTC): 2025-12-25T20:16:24.130536+00:00


In [15]:
def pick_path(*candidates):
    for p in candidates:
        if os.path.exists(p):
            return p
    raise FileNotFoundError(f"Не нашла файл. Проверила: {candidates}")

ARBUZ_PATH   = pick_path("/content/arbuz_astana_all.csv",   "/mnt/data/arbuz_astana_all.csv")
CLEVER_PATH  = pick_path("/content/clever_astana_all.csv",  "/mnt/data/clever_astana_all.csv")
VKUSMART_PATH= pick_path("/content/vkusmart_astana_full.csv","/mnt/data/vkusmart_astana_full.csv")

paths = {"arbuz": ARBUZ_PATH, "clever": CLEVER_PATH, "vkusmart": VKUSMART_PATH}
paths

{'arbuz': '/content/arbuz_astana_all.csv',
 'clever': '/content/clever_astana_all.csv',
 'vkusmart': '/content/vkusmart_astana_full.csv'}

In [16]:
def read_csv_safely(path: str) -> pd.DataFrame:
    for enc in ["utf-8", "utf-8-sig", "cp1251"]:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    return pd.read_csv(path)

raw = {k: read_csv_safely(p) for k, p in paths.items()}

for k, df in raw.items():
    print(k, df.shape)
    display(df.head(2))
print("TOTAL rows:", sum(len(df) for df in raw.values()))

arbuz (957, 11)


,store,city,category_name,category_id,product_id,product_name,brand,price_kzt,currency,unit,source
0,arbuz,astana,Новый год,224993,253199,Приправа Kotanyi для греческого салата 13 г,Kotanyi,500,KZT,NaN,arbuz.kz
1,arbuz,astana,Новый год,224993,253201,Приправа Kotanyi для салата Цезарь 13 г,Kotanyi,555,KZT,NaN,arbuz.kz


clever (1305, 11)


,store,city,category_name,category_id,product_id,product_name,brand,price_kzt,currency,unit,source
0,clever,astana,"Фрукты, ягоды",1090,27f09269-d9f4-11ec-b2bc-005056a59fae,Бананы кг,NaN,1190,KZT,NaN,clevermarket.kz
1,clever,astana,"Фрукты, ягоды",1090,164595af-da0c-11ec-b2bc-005056a59fae,Яблоки Лимонка кг,NaN,940,KZT,NaN,clevermarket.kz


vkusmart (1034, 12)


,store,city,category_name,category_url,product_id,product_name,brand,price_kzt,currency,unit,source,scraped_at
0,vkusmart,astana,Подарочные корзины,https://vkusmart.vmv.kz/catalog/podarochnye-ko...,180238,Корзина Подарочная арт20000,NaN,20000,KZT,/шт,vkusmart.vmv.kz,2025-12-24T18:50:39.449197+00:00
1,vkusmart,astana,Подарочные корзины,https://vkusmart.vmv.kz/catalog/podarochnye-ko...,184073,Корзина Подарочная арт40000,NaN,40000,KZT,/шт,vkusmart.vmv.kz,2025-12-24T18:50:39.449197+00:00


TOTAL rows: 3296


In [17]:
change_log = []

def log_event(step, dataset, metric, value, details=""):
    change_log.append({
        "run_at_utc": RUN_AT_UTC,
        "step": step,
        "dataset": dataset,
        "metric": metric,
        "value": int(value) if isinstance(value, (int, np.integer)) else value,
        "details": details
    })

def normalize_text(x):
    if pd.isna(x):
        return np.nan
    s = str(x).replace("\u00a0", " ")
    s = s.replace("ё", "е").replace("Ё", "Е")
    s = re.sub(r"\s+", " ", s).strip()
    return s

In [18]:
REQUIRED_COLS = ["store","city","category_name","category_id","product_id","product_name","brand","price_kzt","currency","unit","source","scraped_at"]

def standardize_schema(df: pd.DataFrame, dataset: str) -> pd.DataFrame:
    df = df.copy()
    df.columns = [c.strip() for c in df.columns]

    # Добавим недостающие колонки
    for c in REQUIRED_COLS:
        if c not in df.columns:
            df[c] = np.nan
            log_event("add_missing_column", dataset, "added_col", 1, f"Added: {c}")

    # Нормализация текста
    text_cols = ["store","city","category_name","product_name","brand","currency","unit","source"]
    for c in text_cols:
        before_na = df[c].isna().sum()
        df[c] = df[c].apply(normalize_text)
        after_na = df[c].isna().sum()
        log_event("normalize_text", dataset, f"na_{c}_before", before_na)
        log_event("normalize_text", dataset, f"na_{c}_after", after_na)

    # Типы
    df["product_id"] = pd.to_numeric(df["product_id"], errors="coerce").astype("Int64")
    df["category_id"] = pd.to_numeric(df["category_id"], errors="coerce").astype("Int64")
    df["price_kzt"] = pd.to_numeric(df["price_kzt"], errors="coerce")

    # scraped_at (у vkusmart есть, у других может быть пусто)
    df["scraped_at"] = pd.to_datetime(df["scraped_at"], errors="coerce", utc=True)

    # Служебные поля
    df["dataset"] = dataset
    df["pipeline_run_at_utc"] = RUN_AT_UTC

    # Хэш строки (ВАЖНО: UInt64, чтобы не падало)
    df["row_hash_raw"] = pd.util.hash_pandas_object(
        df[["store","city","category_name","category_id","product_id","product_name","brand","price_kzt","currency","unit","source"]],
        index=False
    ).astype("UInt64")

    log_event("standardize_schema", dataset, "rows", len(df))
    return df

std = {k: standardize_schema(df, k) for k, df in raw.items()}

In [19]:
def quality_report(df: pd.DataFrame, dataset: str):
    log_event("quality", dataset, "rows", len(df))
    log_event("quality", dataset, "dup_full_rows", df.duplicated().sum())
    # “мягкие” дубликаты по ключам — только анализируем, НЕ удаляем
    log_event("quality", dataset, "dup_store_product_id", df.duplicated(["store","product_id"]).sum())
    log_event("quality", dataset, "missing_price", df["price_kzt"].isna().sum())
    log_event("quality", dataset, "missing_unit", df["unit"].isna().sum())
    log_event("quality", dataset, "missing_brand", df["brand"].isna().sum())

for k, df in std.items():
    quality_report(df, k)

pd.DataFrame(change_log).query("step=='quality'")

,run_at_utc,step,dataset,metric,value,details
54,2025-12-25T20:16:24.130536+00:00,quality,arbuz,rows,957,
55,2025-12-25T20:16:24.130536+00:00,quality,arbuz,dup_full_rows,0,
56,2025-12-25T20:16:24.130536+00:00,quality,arbuz,dup_store_product_id,80,
57,2025-12-25T20:16:24.130536+00:00,quality,arbuz,missing_price,0,
58,2025-12-25T20:16:24.130536+00:00,quality,arbuz,missing_unit,957,
59,2025-12-25T20:16:24.130536+00:00,quality,arbuz,missing_brand,91,
60,2025-12-25T20:16:24.130536+00:00,quality,clever,rows,1305,
61,2025-12-25T20:16:24.130536+00:00,quality,clever,dup_full_rows,0,
62,2025-12-25T20:16:24.130536+00:00,quality,clever,dup_store_product_id,1304,
63,2025-12-25T20:16:24.130536+00:00,quality,clever,missing_price,0,


In [20]:
UNIT_RE = re.compile(r"(?P<qty>\d+(?:[.,]\d+)?)\s*(?P<u>кг|г|гр|л|мл|шт|уп|пак)\b", flags=re.IGNORECASE)

def extract_unit_from_name(name):
    if pd.isna(name):
        return (np.nan, np.nan)
    m = UNIT_RE.search(str(name))
    if not m:
        return (np.nan, np.nan)
    qty = float(m.group("qty").replace(",", "."))
    u = m.group("u").lower()
    if u == "гр": u = "г"
    return (qty, u)

def clean_dataset(df: pd.DataFrame, dataset: str) -> pd.DataFrame:
    df = df.copy()

    # 1) price: <=0 -> NaN (ошибка данных)
    bad_price = df["price_kzt"].notna() & (df["price_kzt"] <= 0)
    log_event("clean_price", dataset, "bad_price_count", bad_price.sum())
    df.loc[bad_price, "price_kzt"] = np.nan

    # 2) currency: если пусто, ставим KZT (так как поле price_kzt)
    miss_cur = df["currency"].isna().sum()
    df["currency"] = df["currency"].fillna("KZT")
    log_event("fill_currency", dataset, "filled_currency", miss_cur)

    # 3) unit: пробуем достать из product_name, иначе оставляем "unknown"
    qty_unit = df["product_name"].apply(extract_unit_from_name)
    df["pack_qty"] = qty_unit.apply(lambda x: x[0])
    df["pack_unit"] = qty_unit.apply(lambda x: x[1])

    miss_unit_before = df["unit"].isna().sum()
    df["unit"] = df["unit"].fillna(df["pack_unit"])
    df["unit"] = df["unit"].fillna("unknown")
    miss_unit_after = (df["unit"] == "unknown").sum()
    log_event("fill_unit", dataset, "missing_unit_before", miss_unit_before)
    log_event("fill_unit", dataset, "unknown_unit_after", miss_unit_after)

    # 4) brand: если пусто -> Unknown
    miss_brand = df["brand"].isna().sum()
    df["brand"] = df["brand"].fillna("Unknown")
    log_event("fill_brand", dataset, "filled_brand", miss_brand)

    # 5) Удаляем ТОЛЬКО полные дубликаты (их у тебя фактически 0, но по заданию делаем)
    dup_full = df.duplicated().sum()
    df = df.drop_duplicates(keep="first")
    log_event("drop_full_duplicates", dataset, "removed", dup_full)

    # ВАЖНО: НЕ делаем groupby(store, product_id) и НЕ режем категории!
    return df

cleaned = {k: clean_dataset(df, k) for k, df in std.items()}

for k, df in cleaned.items():
    print(k, df.shape)
print("TOTAL after cleaning:", sum(len(df) for df in cleaned.values()))

arbuz (957, 17)
clever (1305, 17)
vkusmart (1034, 18)
TOTAL after cleaning: 3296


In [21]:
# Выбросы по 1.5×IQR (исправление: capping + флаг)
def iqr_bounds(s: pd.Series):
    s = pd.to_numeric(s, errors="coerce").dropna()
    if len(s) < 20:
        return (np.nan, np.nan)
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    return (float(q1 - 1.5*iqr), float(q3 + 1.5*iqr))

def apply_iqr_capping_by_store(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["price_kzt_is_outlier_iqr"] = False
    df["price_kzt_clean"] = df["price_kzt"]

    for store, idx in df.groupby("store").groups.items():
        lo, hi = iqr_bounds(df.loc[idx, "price_kzt"])
        if np.isnan(lo) or np.isnan(hi):
            continue
        s = df.loc[idx, "price_kzt"]
        out = s.notna() & ((s < lo) | (s > hi))
        df.loc[idx, "price_kzt_is_outlier_iqr"] = out
        df.loc[idx, "price_kzt_clean"] = s.clip(lower=max(0, lo), upper=hi)

    return df

combined = pd.concat([apply_iqr_capping_by_store(df) for df in cleaned.values()], ignore_index=True)

print("COMBINED:", combined.shape)
print("Outliers flagged:", combined["price_kzt_is_outlier_iqr"].sum())

COMBINED: (3296, 20)
Outliers flagged: 310


In [22]:
STALE_DAYS = 7
now_utc = pd.Timestamp.now(tz="UTC")

combined["is_scraped_at_missing"] = combined["scraped_at"].isna()
combined["is_stale"] = False
mask = combined["scraped_at"].notna()
combined.loc[mask, "is_stale"] = (now_utc - combined.loc[mask, "scraped_at"]) > pd.Timedelta(days=STALE_DAYS)

print("scraped_at missing:", combined["is_scraped_at_missing"].sum())
print("stale:", combined["is_stale"].sum())

scraped_at missing: 2262
stale: 0


In [23]:
final_cols = [
    "store","city","category_name","category_id","product_id","product_name","brand",
    "price_kzt","price_kzt_clean","price_kzt_is_outlier_iqr",
    "currency","unit","pack_qty","pack_unit","source","scraped_at",
    "is_scraped_at_missing","is_stale","dataset","pipeline_run_at_utc"
]
final_df = combined[final_cols].copy()

print("FINAL shape:", final_df.shape)
assert final_df.shape[1] >= 5, "Мало полей (<5)"
assert final_df.shape[0] >= 2500, f"Мало записей (<2500). Сейчас: {final_df.shape[0]}"

display(final_df.head(10))

FINAL shape: (3296, 20)


,store,city,category_name,category_id,product_id,product_name,brand,price_kzt,price_kzt_clean,price_kzt_is_outlier_iqr,currency,unit,pack_qty,pack_unit,source,scraped_at,is_scraped_at_missing,is_stale,dataset,pipeline_run_at_utc
0,arbuz,astana,Новый год,224993,253199,Приправа Kotanyi для греческого салата 13 г,Kotanyi,500.0,500.0,False,KZT,г,13.0,г,arbuz.kz,NaT,True,False,arbuz,2025-12-25T20:16:24.130536+00:00
1,arbuz,astana,Новый год,224993,253201,Приправа Kotanyi для салата Цезарь 13 г,Kotanyi,555.0,555.0,False,KZT,г,13.0,г,arbuz.kz,NaT,True,False,arbuz,2025-12-25T20:16:24.130536+00:00
2,arbuz,astana,Новый год,224993,252534,Чеснок Vegeta Natur гранулированный 15 г,Vegeta,426.0,426.0,False,KZT,г,15.0,г,arbuz.kz,NaT,True,False,arbuz,2025-12-25T20:16:24.130536+00:00
3,arbuz,astana,Новый год,224993,234538,"Хлебцы Здорово гречневые, 100 г",Здорово,585.0,585.0,False,KZT,г,100.0,г,arbuz.kz,NaT,True,False,arbuz,2025-12-25T20:16:24.130536+00:00
4,arbuz,astana,Новый год,224993,19445,"Бананы, кг",Unknown,1165.0,1165.0,False,KZT,unknown,NaN,NaN,arbuz.kz,NaT,True,False,arbuz,2025-12-25T20:16:24.130536+00:00
5,arbuz,astana,Новый год,224993,224493,"Напиток Holiday лимонад газированный, 1 л",Holiday,655.0,655.0,False,KZT,л,1.0,л,arbuz.kz,NaT,True,False,arbuz,2025-12-25T20:16:24.130536+00:00
6,arbuz,astana,Новый год,224993,334212,"Адвент-календарь Riegelein шоколадный, 65 г",Riegelein,1499.0,1499.0,False,KZT,г,65.0,г,arbuz.kz,NaT,True,False,arbuz,2025-12-25T20:16:24.130536+00:00
7,arbuz,astana,Новый год,224993,333216,Подарочный набор Kerasys Proactive Man цитрус ...,Kerasys,9270.0,6558.0,True,KZT,мл,400.0,мл,arbuz.kz,NaT,True,False,arbuz,2025-12-25T20:16:24.130536+00:00
8,arbuz,astana,Новый год,224993,333217,Подарочный набор Kerasys Proactive Man лайм ша...,Kerasys,9270.0,6558.0,True,KZT,мл,400.0,мл,arbuz.kz,NaT,True,False,arbuz,2025-12-25T20:16:24.130536+00:00
9,arbuz,astana,Новый год,224993,334643,Дед Мороз музыкальный,Unknown,15875.0,6558.0,True,KZT,unknown,NaN,NaN,arbuz.kz,NaT,True,False,arbuz,2025-12-25T20:16:24.130536+00:00


In [24]:
log_df = pd.DataFrame(change_log)

out_csv = "products_merged_clean.csv"
out_xml = "products_merged_clean.xml"
out_log = "cleaning_change_log.csv"

final_df.to_csv(out_csv, index=False, encoding="utf-8-sig")

# XML любит простые типы — datetime ок, списков у нас нет
final_df.to_xml(out_xml, index=False, root_name="products", row_name="product")

log_df.to_csv(out_log, index=False, encoding="utf-8-sig")

print("Saved:", out_csv, out_xml, out_log)

Saved: products_merged_clean.csv products_merged_clean.xml cleaning_change_log.csv


In [25]:
from google.colab import files
files.download(out_csv)
files.download(out_xml)
files.download(out_log)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>